<a href="https://colab.research.google.com/github/brady-at-claradata/FinRL/blob/master/PPO-Stock-Trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Stock Trading Application in ElegantRL**






In [1]:
import torch

In [2]:
torch.cuda.get_device_properties(0)


_CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)

# **Part 1: Problem Formulation**
Formally, we model stock trading as a Markov Decision Process (MDP), and formulate the trading objective as maximization of expected return:



*   **State s = [b, p, h]**: a vector that includes the remaining balance b, stock prices p, and stock shares h. p and h are vectors with D dimension, where D denotes the number of stocks. 
*   **Action a**: a vector of actions over D stocks. The allowed actions on each stock include selling, buying, or holding, which result in decreasing, increasing, or no change of the stock shares in h, respectively.
*   **Reward r(s, a, s’)**: The asset value change of taking action a at state s and arriving at new state s’.
*   **Policy π(s)**: The trading strategy at state s, which is a probability distribution of actions at state s.
*   **Q-function Q(s, a)**: the expected return (reward) of taking action a at state s following policy π.
*   **State-transition**: After taking the actions a, the number of shares h is modified, as shown in Fig 3, and the new portfolio is the summation of the balance and the total value of the stocks.

# **Part 2: Stock Trading Environment Design**

**State Space and Action Space**


*   **State Space**: We use a 181-dimensional vector consists of seven parts of information to represent the state space of multiple stocks trading environment: [b, p, h, M, R, C, X], where b is the balance, p is the stock prices, h is the number of shares, M is the Moving Average Convergence Divergence (MACD), R is the Relative Strength Index (RSI), C is the Commodity Channel Index (CCI), and X is the Average Directional Index (ADX).
*   **Action Space**: As a recap, we have three types of actions: selling, buying, and holding for a single stock. We use the negative value for selling, positive value for buying, and zero for holding. In this case, the action space is defined as {-k, …, -1, 0, 1, …, k}, where k is the maximum share to buy or sell in each transaction.


**Easy-to-customize Features**


*   **initial_capital**: the initial capital that the user wants to invest.
*   **tickers**: the stocks that the user wants to trade with.
*   **initial_stocks**: the initial amount of each stock and the default could be zero.
*   **buy_cost_pct, sell_cost_pct**: the transaction fee of each buying or selling transaction.
*   **max_stock**: the user is able to define the maximum number of stocks that are allowed to trade per transaction. Users can also set the maximum percentage of capitals to invest in each stock.
*   **tech_indicator_list**: the list of financial indicators that are taken into account, which is used to define a state.
*   **start_date, start_eval_date, end_eval_date**: the training and backtesting time intervals. Thee time dates (or timestamps) are used, once the training period is specified, the rest is backtesting.


# **Part 3: Install ElegantRL and related packages**

In [3]:
!pip install git+https://github.com/AI4Finance-LLC/ElegantRL.git
!pip install yfinance stockstats

  Cloning https://github.com/AI4Finance-LLC/ElegantRL.git to /tmp/pip-req-build-wy6f1idg
  Running command git clone -q https://github.com/AI4Finance-LLC/ElegantRL.git /tmp/pip-req-build-wy6f1idg


# **Part 4: Import Packages**


*   **ElegantRL**
*   **yfinance**: yfinance aims to solve this problem by offering a reliable, threaded, and Pythonic way to download historical market data from Yahoo! finance.
*   **StockDataFrame**: stockstats inherits and extends pandas.DataFrame to support Stock Statistics and Stock Indicators.



In [6]:
from elegantrl.run import *
from elegantrl.agent import *
from elegantrl.envs.FinRL.StockTrading import StockTradingEnv, check_stock_trading_env
import yfinance as yf
from stockstats import StockDataFrame as Sdf
from datetime import datetime, timedelta

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Part 5: Specify Agent and Environment**

*   **args.agent**: firstly chooses one DRL algorithm to use from agent.py. In this application, we prefer to choose DDPG and PPO agent.
*   **args.env**: creates the environment, and the user can either customize own environment or preprocess environments from OpenAI Gym and PyBullet Gym from env.py. In this application, we create the self-designed stock trading environment.


> Before finishing initialization of **args**, please see Arguments() in run.py for more details about adjustable hyper-parameters.




In [9]:
# Agent
args = Arguments(if_on_policy=True)
args.agent = AgentPPO() # AgentSAC(), AgentTD3(), AgentDDPG()
args.agent.if_use_gae = True
args.agent.lambda_entropy = 0.04

# Environment
tickers = [
  'AAPL', 'ADBE', 'ADI', 'ADP', 'ADSK', 'ALGN', 'ALXN', 'AMAT', 'AMD', 'AMGN',
  'AMZN', 'ASML', 'ATVI', 'BIIB', 'BKNG', 'BMRN', 'CDNS', 'CERN', 'CHKP', 'CMCSA',
  'COST', 'CSCO', 'CSX', 'CTAS', 'CTSH', 'CTXS', 'DLTR', 'EA', 'EBAY', 'FAST',
  'FISV', 'GILD', 'HAS', 'HSIC', 'IDXX', 'ILMN', 'INCY', 'INTC', 'INTU', 'ISRG',
  'JBHT', 'KLAC', 'LRCX', 'MAR', 'MCHP', 'MDLZ', 'MNST', 'MSFT', 'MU', 'MXIM',
  'NLOK', 'NTAP', 'NTES', 'NVDA', 'ORLY', 'PAYX', 'PCAR', 'PEP', 'QCOM', 'REGN',
  'ROST', 'SBUX', 'SIRI', 'SNPS', 'SWKS', 'TTWO', 'TXN', 'VRSN', 'VRTX', 'WBA',
  'WDC', 'WLTW', 'XEL', 'XLNX']  # finrl.config.NAS_74_TICKER

tech_indicator_list = [
  'macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30',
  'close_30_sma', 'close_60_sma']  # finrl.config.TECHNICAL_INDICATORS_LIST

gamma = 0.99
max_stock = 1e2
initial_capital = 1e6
initial_stocks = np.zeros(len(tickers), dtype=np.float32)
buy_cost_pct = 0.
sell_cost_pct = 0.
start_date = '2008-03-19'
start_eval_date = (datetime.now() - timedelta(days=365*2)).strftime("%F")
end_eval_date = datetime.now().strftime('%F')

args.env = StockTradingEnv('./', gamma, max_stock, initial_capital, buy_cost_pct, 
                           sell_cost_pct, start_date, start_eval_date, 
                           end_eval_date, tickers, tech_indicator_list, 
                           initial_stocks, if_eval=False)
args.env_eval = StockTradingEnv('./', gamma, max_stock, initial_capital, buy_cost_pct, 
                           sell_cost_pct, start_date, start_eval_date, 
                           end_eval_date, tickers, tech_indicator_list, 
                           initial_stocks, if_eval=True)

args.env.target_reward = 3
args.env_eval.target_reward = 3

# Hyperparameters
args.gamma = gamma
args.break_step = int(5e5)
args.net_dim = 2 ** 9
args.max_step = args.env.max_step
args.max_memo = args.max_step * 4
args.batch_size = 2 ** 9
args.repeat_times = 2 ** 3
args.eval_gap = 2 ** 4
args.eval_times1 = 2 ** 3
args.eval_times2 = 2 ** 5
args.if_allow_break = False
args.rollout_num = 2 # the number of rollout workers (larger is not always faster)

| FeatureEngineer: start processing data (2 minutes)
| YahooDownloader: start downloading data (1 minute)
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 complet

In [14]:
train_and_evaluate(args) # the training process will terminate once it reaches the target reward.

| GPU id: 0, cwd: ./AgentPPO/StockTradingEnv-v1_0
ID      Step      MaxR |    avgR      stdR       objA      objC |  avgS  stdS
0   4.34e+03      1.45 |
0   4.34e+03      1.45 |    1.45      0.01      -0.16      1.48 |   362     0
0   2.61e+04      1.60 |
0   2.61e+04      1.60 |    1.60      0.02      -0.22      0.57 |   362     0
0   4.78e+04      2.11 |
0   4.78e+04      2.11 |    2.11      0.02      -0.07      0.54 |   362     0
0   6.95e+04      2.47 |
0   6.95e+04      2.47 |    2.47      0.02      -0.11      0.39 |   362     0
0   9.12e+04      2.51 |
0   9.12e+04      2.51 |    2.51      0.02      -0.08      0.72 |   362     0
0   1.13e+05      2.51 |    2.15      0.06      -0.03      0.40 |   362     0
0   1.35e+05      2.69 |
0   1.35e+05      2.69 |    2.69      0.07      -0.13      0.70 |   362     0
0   1.56e+05      3.03 |
0   1.56e+05      3.03 |    3.03      0.04      -0.09      0.40 |   362     0
0   1.78e+05      3.44 |
0   1.78e+05      3.44 |    3.44      0.06      

In [12]:
!cp -R /content/AgentPPO/ /content/drive/MyDrive/trading/

Understanding the above results::
*   **Step**: the total training steps.
*  **MaxR**: the maximum reward.
*   **avgR**: the average of the rewards.
*   **stdR**: the standard deviation of the rewards.
*   **objA**: the objective function value of Actor Network (Policy Network).
*   **objC**: the objective function value (Q-value)  of Critic Network (Value Network).

# **Part 7: Backtest and Draw the Graph**

In [86]:
args = Arguments(if_on_policy=True)
# args.agent = AgentPPO() # AgentSAC(), AgentTD3(), AgentDDPG()
# args.agent.if_use_gae = True
# args.agent.lambda_entropy = 0.04

# Environment
tickers = [
  'AAPL', 'ADBE', 'ADI', 'ADP', 'ADSK', 'ALGN', 'ALXN', 'AMAT', 'AMD', 'AMGN',
  'AMZN', 'ASML', 'ATVI', 'BIIB', 'BKNG', 'BMRN', 'CDNS', 'CERN', 'CHKP', 'CMCSA',
  'COST', 'CSCO', 'CSX', 'CTAS', 'CTSH', 'CTXS', 'DLTR', 'EA', 'EBAY', 'FAST',
  'FISV', 'GILD', 'HAS', 'HSIC', 'IDXX', 'ILMN', 'INCY', 'INTC', 'INTU', 'ISRG',
  'JBHT', 'KLAC', 'LRCX', 'MAR', 'MCHP', 'MDLZ', 'MNST', 'MSFT', 'MU', 'MXIM',
  'NLOK', 'NTAP', 'NTES', 'NVDA', 'ORLY', 'PAYX', 'PCAR', 'PEP', 'QCOM', 'REGN',
  'ROST', 'SBUX', 'SIRI', 'SNPS', 'SWKS', 'TTWO', 'TXN', 'VRSN', 'VRTX', 'WBA',
  'WDC', 'WLTW', 'XEL', 'XLNX']  # finrl.config.NAS_74_TICKER

tech_indicator_list = [
  'macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30',
  'close_30_sma', 'close_60_sma']  # finrl.config.TECHNICAL_INDICATORS_LIST

gamma = 0.99
max_stock = 1e2
initial_capital = 1e6 #TODO SET
initial_stocks = np.zeros(len(tickers), dtype=np.float32) #TODO SET
buy_cost_pct = 0.
sell_cost_pct = 0.
start_date = '2008-03-19'
start_eval_date = (datetime.now() - timedelta(days=365*2)).strftime("%F")
end_eval_date = datetime.now().strftime('%F')
#get you data!!
args.env = StockTradingEnv('./', gamma, max_stock, initial_capital, buy_cost_pct, 
                           sell_cost_pct, start_date, start_eval_date, 
                           end_eval_date, tickers, tech_indicator_list, 
                           initial_stocks, if_eval=True)

args.agent = AgentPPO()

args.if_remove = False
args.cwd = './drive/MyDrive/trading/AgentPPO/StockTradingEnv-v1_0'
# args.cwd = './AgentPPO/StockTradingEnv-v1_0'

args.init_before_training()

# args.env.draw_cumulative_return(args, torch)

| GPU id: 0, cwd: ./drive/MyDrive/trading/AgentPPO/StockTradingEnv-v1_0


In [91]:
state_dim = args.env.state_dim
action_dim = args.env.action_dim

agent = args.agent
net_dim = args.net_dim
cwd = args.cwd


agent.init(net_dim, state_dim, action_dim)
agent.save_load_model(cwd=cwd, if_save=False)
act = agent.act
device = agent.device

state = np.hstack((args.env.initial_capital * 2 ** -13,
  args.env.price_ary[args.env.max_step],
  args.env.initial_stocks,
  args.env.tech_ary[args.env.max_step],)).astype(np.float32) * 2 ** -5

episode_returns = list()  # the cumulative_return / initial_account
action = None
with torch.no_grad():
    s_tensor = torch.as_tensor((state,), device=device)
    a_tensor = args.agent.act(s_tensor)
    action = a_tensor.cpu().numpy()[0]  # not need detach(), because with torch.no_grad() outside
    
    # state, reward, done, _ = self.step(action) modified step to actually trade

    # total_asset = self.amount + (self.price_ary[self.day] * self.stocks).sum()
    # episode_return = total_asset / self.initial_total_asset
    # episode_returns.append(episode_return)



Loaded act: ./drive/MyDrive/trading/AgentPPO/StockTradingEnv-v1_0


In [16]:
!rm -rf /content/drive/MyDrive/trading/AgentPPO/ 
!cp -R ./AgentPPO /content/drive/MyDrive/trading/

In [95]:
args.env.initial_stocks

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.], dtype=float32)

In [102]:
actions = (action * args.env.max_stock).astype(int)

price = args.env.price_ary[args.env.max_step]

start_asset = args.env.initial_capital + (args.env.initial_stocks * price).sum()

for index in np.where(actions < 0)[0]:  # sell_index:
    if price[index] > 0:  # Sell only if current asset is > 0
        sell_num_shares = min(args.env.initial_stocks[index], -actions[index])
        args.env.initial_stocks[index] -= sell_num_shares
        args.env.initial_capital += price[index] * sell_num_shares * (1 - args.env.sell_cost_pct)

for index in np.where(actions > 0)[0]:  # buy_index:
    if price[index] > 0:  # Buy only if the price is > 0 (no missing data in this particular date)
        buy_num_shares = min(args.env.initial_capital // price[index], actions[index])
        args.env.initial_stocks[index] += buy_num_shares
        args.env.initial_capital -= price[index] * buy_num_shares * (1 + args.env.buy_cost_pct)

state = np.hstack((args.env.initial_capital * 2 ** -13,
  args.env.price_ary[args.env.max_step],
  args.env.initial_stocks,
  args.env.tech_ary[args.env.max_step],)).astype(np.float32) * 2 ** -5

total_asset = args.env.initial_capital + (args.env.initial_stocks * price).sum()
print(start_asset, total_asset)

1000000.0 1000000.00390625
